# Inicialización

## Entorno (Pre-Requisitos)

In [1]:
%pip install unidecode utm pandas numpy matplotlib seaborn joblib scikit-learn ipykernel Jinja2 sentence-transformers ipywidgets unidecode GPUtil

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # Para gráficos estadísticos
from jinja2 import Environment, BaseLoader # Para generación de informes
import os
import re
import unidecode

from sentence_transformers import SentenceTransformer, util
import utm


from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.dummy import DummyRegressor

## Constantes

In [3]:
# Indicador para saber si se está trabajando en Google Drive
is_Drive = False

# Ruta base donde se guardarán o cargarán los archivos
base_path = ''
img_folder_path = "Informe/img"
tex_folder_path = "Informe/tex"

# Si estamos trabajando en Google Drive, ejecutar este bloque
if (is_Drive):
    from google.colab import drive, files # type: ignore
    drive.mount('/content/drive/', force_remount=True)
    base_path = '/content/drive/MyDrive/Machine_Learning_Para_Sistemas_Inteligentes/'

In [4]:
target_luxury = (
    "luxury luxe lujo premium exclusive exclusivo high-end alta gama "
    "boutique designer diseño exklusiv "
    "lavish opulent elegant posh prestigious affluent sophisticated "
    "sumptuous elite grand luxurious palatial glamorous chic refined stylish "
    "vintage 'designer brand' 'exclusive access'"
)

target_studio = (
    "studio estudio monoambiente loft microstudio small flat tiny apartment "
    "compact living microapartamento 'espacio reducido' 'estudio funcional' "
    "estudio compacto 'apartamento pequeño' 'mini loft' 'smart apartment' "
    "minimalista 'apartamento de diseño' 'smart living' 'living space' "
    "open-plan 'one-bedroom apartment' 'space-saving' compact apartment "
    "cozy apartment 'minimalist studio'"
)

target_room = (
    "room habitacion quarto chambre cuarto 'private room' 'shared room' "
    "small room dormitorio habitación habitáculo 'habitación privada' "
    "'habitación compartida' estancia 'cuarto pequeño' suite bedroom "
    "'room with a view' 'single room' 'shared space' 'personal space' "
    "'living quarters' 'guest room' 'room for rent' 'co-living space' "
    "'private suite'"
)

## Métodos Auxiliares

### Load

In [5]:
def explorar_df(df):
    """Imprime shape, head, info, describe e información de nulos de un DataFrame."""
    print(df.shape)
    display(df.head())
    print(df.info())
    print(df.describe())
    print(df.isnull().sum())

### Preprocesamiento

In [6]:
def distancia(lati1, lon1, lat2, lon2):
    """
    Calcula la distancia euclidiana en metros entre puntos usando proyección UTM.
    Soporta inputs escalares o Series/Listas de coordenadas.
    lat2, lon2 se asumen como el punto de referencia (escalar).
    """
    # Convertir punto de referencia (landmark) a UTM
    x_2, y_2 = utm.from_latlon(lat2, lon2)[:2]
    
    # Verificar si lati1 es iterable (Series, array, lista)
    if hasattr(lati1, '__iter__') and not isinstance(lati1, str):
        # Iterar sobre las coordenadas para convertir a UTM una por una
        utm_coords = [utm.from_latlon(lat, lon)[:2] for lat, lon in zip(lati1, lon1)]
        
        # Separar en arrays numpy para cálculo vectorizado de distancia
        x_1 = np.array([c[0] for c in utm_coords])
        y_1 = np.array([c[1] for c in utm_coords])
    else:
        # Caso escalar
        x_1, y_1 = utm.from_latlon(lati1, lon1)[:2]
        
    return np.sqrt((x_2 - x_1) ** 2 + (y_2 - y_1) ** 2)

def IQR_outlier_removal(df, columns):
    for col in columns:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            # Recortamos los valores extremos al rango [lower, upper]
            df[col] = df[col].clip(lower=lower, upper=upper)
    return df

def winsorize_outlier_removal(df, columns, lower_percentile=0.01, upper_percentile=0.99):
    for col in columns:
        if col in df.columns:
            lower = df[col].quantile(lower_percentile)
            upper = df[col].quantile(upper_percentile)
            df[col] = df[col].clip(lower=lower, upper=upper)
    return df

In [7]:
def preprocess_data(df, price_map=None):
    
    # 1. Tratamiento de valores faltantes
    df["never_reviewed"] = df["last_review"].isna().astype(int)
    df["reviews_per_month"] = df["reviews_per_month"].fillna(0)
    
    # Convertir last_review a datetime y crear days_since_last_review
    df["last_review"] = pd.to_datetime(df["last_review"], errors="coerce")
    df["days_since_last_review"] = (pd.Timestamp("today") - df["last_review"]).dt.days
    df["days_since_last_review"] = df["days_since_last_review"].fillna(df["days_since_last_review"].max())

    # 2. Features NLP mejoradas
    df["name"] = df["name"].fillna("").astype(str)
    df['name_clean'] = df['name'].str.lower().apply(unidecode.unidecode)

    # NPL
    # Se intenta identificar conceptos de lujo, studio y room mediante embeddings
    model_st = SentenceTransformer('all-MiniLM-L6-v2')

    name_embeddings = model_st.encode(df['name_clean'].tolist(), convert_to_tensor=True)

    emb_luxury =  model_st.encode(target_luxury, convert_to_tensor=True)
    emb_studio = model_st.encode(target_studio, convert_to_tensor=True)
    emb_room = model_st.encode(target_room, convert_to_tensor=True)

    luxury_sim = util.cos_sim(name_embeddings, emb_luxury).cpu().numpy().flatten()
    studio_sim = util.cos_sim(name_embeddings, emb_studio).cpu().numpy().flatten()
    room_sim = util.cos_sim(name_embeddings, emb_room).cpu().numpy().flatten()

    df["sim_luxury"] = luxury_sim.astype(float)
    df["sim_studio"] = studio_sim.astype(float)
    df["sim_room"] = room_sim.astype(float)


    # 3. Transformaciones log
    df["minimum_nights_log"] = np.log1p(df["minimum_nights"])
    df["number_of_reviews_log"] = np.log1p(df["number_of_reviews"])
    df["availability_365_log"] = np.log1p(df["availability_365"])
    df["calculated_host_listings_count_log"] = np.log1p(df["calculated_host_listings_count"])
    
    # Solo aplicar log al precio si existe (en train)
    if "price" in df.columns:
        df["price_log"] = np.log1p(df["price"])

    # 4. Distancia a puntos de interés 
    obelisco_lat, obelisco_lon = -34.6037, -58.3816

    df["distance_to_obelisco"] = distancia(df["latitude"], df["longitude"], obelisco_lat, obelisco_lon)

    # 5. Precio por media del barrio
    if price_map is None:
        if "price" in df.columns:
            # Calcular el precio medio por barrio
            precio_medio_barrio = df.groupby('neighbourhood')['price'].mean().to_dict()
            # Aplicar log
            precio_medio_barrio = {k: np.log1p(v) for k, v in precio_medio_barrio.items()}
        else:
            # Fallback si no hay precio ni mapa (no debería ocurrir en flujo correcto)
            precio_medio_barrio = {}
    else:
        precio_medio_barrio = price_map

    # Mapear el precio medio a cada fila del DataFrame
    df['precio_medio_barrio'] = df['neighbourhood'].map(precio_medio_barrio)
    
    # Rellenar nulos (barrios nuevos en test) con la media global del mapa
    if len(precio_medio_barrio) > 0:
        global_mean = np.mean(list(precio_medio_barrio.values()))
        df['precio_medio_barrio'] = df['precio_medio_barrio'].fillna(global_mean)

    # 6. Eliminación de columnas innecesarias
    df = df.drop(columns=["id", "name", "name_clean", "host_name", "host_id", "price", "minimum_nights",
                          "number_of_reviews", "availability_365", "calculated_host_listings_count"],
                 errors='ignore')

    return df, precio_medio_barrio

### Outliers

In [8]:
# ---------------------------
# Formateo numérico simplificado
# ---------------------------
def smart_format(v):
    if pd.isna(v):
        return ""
    s = f"{v:.4f}"
    entero, dec = s.split(".")
    dec = dec.rstrip("0")
    return entero if dec == "" else f"{entero}.{dec}"

# ---------------------------
# Escapes LaTeX simplificados
# ---------------------------
def latex_escape_header(s):
    s = str(s).replace("_", r"\_\allowbreak ")
    return fr"\texttt{{{s}}}"

def latex_escape_row_label(s):
    return str(s).replace("_", r"\_").replace("%", r"\%")

In [9]:
# ---------------------------
# Generador de tabla LaTeX
# ---------------------------
def generar_tabla_descriptiva_latex(
    df,
    cols,
    percentiles,
    output_path,
    filename,
    caption,
    label,
    incluir_varianza=True
):
    os.makedirs(output_path, exist_ok=True)

    # Filtrar columnas que existen en el DataFrame
    valid_cols = [c for c in cols if c in df.columns]
    
    if len(valid_cols) < len(cols):
        missing = set(cols) - set(valid_cols)
        print(f"[INFO] Columna {filename} (no existen en el DF): {missing}")
    


    # describe()
    desc = df[valid_cols].describe(percentiles=percentiles)

    # varianza opcional
    if incluir_varianza:
        variance = df[valid_cols].var().rename("variance")
        desc = pd.concat([desc, variance.to_frame().T])

    # encabezados
    cols_escaped = [latex_escape_header(c) for c in desc.columns]

    # filas
    rows = [
        (
            latex_escape_row_label(idx),
            [smart_format(desc.loc[idx, c]) for c in desc.columns]
        )
        for idx in desc.index
    ]

    # plantilla LaTeX simplificada
    template_str = r"""
\begin{table}[H]
    \centering
    \setlength{\tabcolsep}{3pt}
    \begin{scriptsize}
    \begin{adjustbox}{width=\textwidth}
    \begin{tabular}{l{% for _ in cols %}r{% endfor %}}
        \toprule
        & {% for col in cols %}{{ col }}{% if not loop.last %} & {% endif %}{% endfor %} \\
        \midrule
        {% for row_label, values in rows %}
        {{ row_label }} & {% for val in values %}{{ val }}{% if not loop.last %} & {% endif %}{% endfor %} \\
        {% endfor %}
        \bottomrule
    \end{tabular}
    \end{adjustbox}
    \end{scriptsize}
    \caption{ {{ caption }} }
    \label{ {{ label }} }
\end{table}
"""

    template = Environment(loader=BaseLoader(), autoescape=False).from_string(template_str)
    latex_table = template.render(cols=cols_escaped, rows=rows, caption=caption, label=label)

    # elimina espacios extra dentro de \label{}
    latex_table = re.sub(r'\\label\{\s*(.*?)\s*\}', r'\\label{\1}', latex_table)

    filepath = os.path.join(output_path, filename)
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(latex_table)

    print("Archivo LaTeX generado:", filepath)

In [10]:
def generar_histogramas(df, df_name, base_path, folder_name="histogramas", bins=50):

    # Crear path final
    output_dir = os.path.join(base_path, folder_name)
    os.makedirs(output_dir, exist_ok=True)

    print(f"[INFO] Guardando histogramas en: {output_dir}")

    # Seleccionar columnas numéricas
    numeric_cols = df.select_dtypes(include=["number"]).columns

    for col in numeric_cols:
        plt.figure(figsize=(6, 4))
        sns.histplot(df[col], bins=bins, kde=True)
        plt.title(f"Histograma de {col} ({df_name})")

        # Nombre del archivo incluye el nombre del dataframe
        filename = f"{col}_{df_name}_histograma_bins{bins}.png"
        filepath = os.path.join(output_dir, filename)

        plt.savefig(filepath, dpi=300, bbox_inches="tight")
        plt.close()

        print(f"   → Guardado: {filepath}")


In [11]:
def generar_histogramas(df, df_name, base_path, folder_name="histogramas", bins=50):
    
    # Crear path final
    output_dir = os.path.join(base_path, folder_name)
    os.makedirs(output_dir, exist_ok=True)

    for col in df.select_dtypes(include="number").columns:
        plt.figure(figsize=(6, 4))
        sns.histplot(df[col], bins=bins, kde=True)
        plt.title(f"{col} ({df_name})")

        fname = f"{col}_{df_name}_histograma_bins{bins}.png"
        plt.savefig(os.path.join(output_dir, fname), dpi=300, bbox_inches="tight")
        plt.close()

        print("→ Guardado:", os.path.join(output_dir, fname))



def plot_percentiles_all_methods(
    numeric_cols,
    dfs,
    labels,
    percentiles=None,
    output_dir="Informe/img/percentiles"
):
    """
    Genera y guarda gráficos de percentiles para cada columna y cada método.
    El archivo se guarda como:
    img/percentiles/{col}_percentiles_max=YYY_min=XXX.png
    donde YYY y XXX son los percentiles máximo y mínimo evaluados.
    """

    # Crear carpeta si no existe
    os.makedirs(output_dir, exist_ok=True)

    if percentiles is None:
        percentiles = [90, 95, 97, 98, 99, 99.5, 99.9]

    q = [p / 100.0 for p in percentiles]

    for col in numeric_cols:

        if not any(col in df.columns for df in dfs):
            continue

        plt.figure(figsize=(10, 6))

        for df_tmp, label in zip(dfs, labels):

            if col not in df_tmp.columns:
                continue

            series = df_tmp[col].dropna()
            if series.empty:
                continue

            p_vals = series.quantile(q)
            plt.plot(percentiles, p_vals.values, marker="o", label=label)

        # Etiquetas
        plt.xlabel("Percentil")
        plt.ylabel(col)
        plt.title(f"Comparación de métodos en percentiles de {col}")
        plt.xticks(percentiles, [f"{p}%" for p in percentiles])
        plt.grid(True)
        plt.legend()
        plt.tight_layout()

        # Crear filename con el percentil máximo y mínimo (no los valores)
        max_p = max(percentiles)
        min_p = min(percentiles)

        filename = f"{col}_percentiles_max={max_p}_min={min_p}.png"
        # Si quisieras formato flotante fijo:
        # filename = f"{col}_percentiles_max={max_p:.1f}_min={min_p:.1f}.png"

        filepath = os.path.join(output_dir, filename)

        plt.savefig(filepath, dpi=300, bbox_inches="tight")
        plt.close()

        print(f"Guardado: {filepath}")


### Train

In [12]:
def export_df_to_latex(
    df,
    filename,
    caption,
    label,
    base_path="Informe/tex",
    folder_name="tables",
    float_format="%.4f",
    big=False
):

    # Crear directorio final
    output_dir = os.path.join(base_path, folder_name)
    os.makedirs(output_dir, exist_ok=True)

    filepath = os.path.join(output_dir, filename)

    # Convertir a LaTeX con escaping seguro
    tex_body = df.to_latex(
        index=False,
        float_format=float_format,
        escape=True,     # evita errores con "_"
        caption=None,
        label=None
    )

    # Plantillas simplificadas
    if not big:
        template = rf"""
\begin{{table}}[H]
\centering
{tex_body}
\caption{{{caption}}}
\label{{{label}}}
\end{{table}}
"""
    else:
        template = rf"""
\begin{{table}}[H]
\centering
\setlength{{\tabcolsep}}{{4pt}}
\begin{{scriptsize}}
\begin{{adjustbox}}{{width=\textwidth}}
{tex_body}
\end{{adjustbox}}
\end{{scriptsize}}
\caption{{{caption}}}
\label{{{label}}}
\end{{table}}
"""

    # Guardar archivo
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(template)

    print("→ LaTeX generado:", filepath)


In [13]:
# rf_result = tune_and_report()
#rf_result["cv_summary"]      # DataFrame ordenado por rmse_mean
#rf_result["metrics_val"]     # métricas en validación
#rf_result["best_estimator"]  # mejor pipeline listo para usar/predicción

def tune_and_report(
    name,
    model,
    param_grid,
    preprocessor,
    X_train, y_train,
    X_val, y_val,
    export_latex=False,
    latex_filename=None,
    latex_caption=None,
    latex_label=None,
    latex_dir="Informe/tex/tables",
    bigTable=False
):
    """
    Realiza GridSearchCV, imprime métricas, devuelve resumen y best model.
    Opcionalmente exporta la tabla en LaTeX.
    """

    print(f"\n🔍 Optimizando {name}...\n")

    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model)
    ])

    grid = GridSearchCV(
        estimator=pipe,
        param_grid=param_grid,
        scoring="neg_root_mean_squared_error",
        cv=5,
        n_jobs=-1
    )

    grid.fit(X_train, y_train)

    # ---- CV Results ----
    cv = pd.DataFrame(grid.cv_results_)
    cv["rmse_mean"] = -cv["mean_test_score"]
    cv["rmse_std"] = cv["std_test_score"]
    cv["MSE_mean"] = cv["rmse_mean"] ** 2

    # Selecciona automáticamente las columnas de hiperparámetros
    param_cols = [c for c in cv.columns if c.startswith("param_")]

    summary_cols = param_cols + ["rmse_mean", "rmse_std", "MSE_mean"]
    summary = cv[summary_cols].sort_values("rmse_mean")

    print("👉 Mejor params:", grid.best_params_)
    print("👉 Mejor RMSE (CV):", -grid.best_score_)

    # ---- Métricas en validación ----
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)

    print(f"📌 {name} (validación) -> RMSE={rmse:.3f}, MSE={mse:.3f}, MAE={mae:.3f}, R2={r2:.3f}")

    # ---- Exportar LaTeX (opcional) ----
    if export_latex:
        export_df_to_latex(
            df=summary,
            filename=latex_filename,
            caption=latex_caption,
            label=latex_label,
            base_path=latex_dir,
            folder_name="",
            big=bigTable
        )

    return {
        "name": name,
        "best_estimator": best_model,
        "cv_summary": summary,
        "metrics_val": {
            "rmse": rmse,
            "mse": mse,
            "mae": mae,
            r"R^{2}": r2
        }
    
    }

In [34]:
def cross_validate_models(
    models_dict,
    X,
    y,
    preprocessor=None,
    cv_folds=5,
    export_latex=False,
    latex_filename="cv_results.tex",
    latex_caption="Resultados de validación cruzada (CV).",
    latex_label="tab:cv-all-models"
):
    """
    Ejecuta validación cruzada para múltiples modelos.
    Devuelve un DataFrame con rmse, mse, mae y r2 promedio.
    Si export_latex=True exporta tabla para LaTeX.
    """

    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    cv_results = []

    for name, model in models_dict.items():
        print(f"→ Ejecutando CV para: {name}")

        # Si ya viene como Pipeline, lo usamos tal cual
        # Si no, lo envolvemos con el preprocessor
        pipe = model if isinstance(model, Pipeline) else Pipeline([
            ("preprocessor", preprocessor),
            ("model", model)
        ])

        # RMSE (negativo de sklearn)
        rmse_scores = -cross_val_score(
            pipe, X, y,
            scoring="neg_root_mean_squared_error",
            cv=kf, n_jobs=-1
        )
        mse_scores = rmse_scores ** 2

        # MAE
        mae_scores = -cross_val_score(
            pipe, X, y,
            scoring="neg_mean_absolute_error",
            cv=kf, n_jobs=-1
        )

        # R²
        r2_scores = cross_val_score(
            pipe, X, y,
            scoring="r2",
            cv=kf, n_jobs=-1
        )

        cv_results.append({
            "Modelo": name,
            "rmse_mean": rmse_scores.mean(),
            "rmse_std": rmse_scores.std(),
            "MSE_mean": mse_scores.mean(),
            "MAE_mean": mae_scores.mean(),
            "R2_mean": r2_scores.mean(),
        })

    df_results = pd.DataFrame(cv_results).sort_values(by="rmse_mean")

    display(df_results)

    # Exportar a LaTeX si corresponde
    if export_latex:
        export_df_to_latex(
            df_results,
            filename=latex_filename,
            caption=latex_caption,
            label=latex_label,
            base_path="Informe/tex",
            folder_name="tables",
            big=True
        )

    return df_results

In [15]:
def evaluar_modelos(models, preprocessor, X_train, y_train, X_val, y_val):

    resultados = []

    for name, model in models.items():
        pipe = Pipeline([
            ("preprocessor", preprocessor),
            ("model", model)
        ])

        try:
            pipe.fit(X_train, y_train)
            y_pred = pipe.predict(X_val)

            mse  = mean_squared_error(y_val, y_pred)
            rmse = np.sqrt(mse)
            mae  = mean_absolute_error(y_val, y_pred)
            r2   = r2_score(y_val, y_pred)

            resultados.append({
                "Modelo": name,
                "RMSE": rmse,
                "MSE": mse,
                "MAE": mae,
                r"R^{2}": r2,
            })

            print(f"{name:<20} RMSE={rmse:7.3f} | MSE={mse:7.3f} | MAE={mae:7.3f} | R2={r2:7.3f}")

        except Exception as e:
            print(f"[ERROR] El modelo '{name}' falló: {e}")
            resultados.append({
                "Modelo": name,
                "RMSE": np.nan,
                "MSE": np.nan,
                "MAE": np.nan,
                r"R^{2}": np.nan,
            })

    return pd.DataFrame(resultados)

# Load Data

In [16]:
print("Cargando datos...")
df_raw_train = pd.read_csv(base_path + "train.csv")
df_raw_test = pd.read_csv(base_path + "test.csv")

Cargando datos...


In [17]:
#info de Train
print("Información sobre train.csv: \n")
explorar_df(df_raw_test)

#Info de Test
print(" \n \n Información sobre test.csv: \n")
explorar_df(df_raw_test)

Información sobre train.csv: 

(6864, 14)


,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,15843708,Monoambiente en Barrio Norte,19787638,Pablo,Recoleta,-34.59053,-58.40898,Entire home/apt,2,26,03-11-2019,0.90,1,74
1,9735218,Busco Roomate :),38507726,Angela,Palermo,-34.58633,-58.41312,Private room,1,0,NaN,NaN,1,0
2,35682605,Betty´s home,100972248,Cecilia,Balvanera,-34.59979,-58.39340,Entire home/apt,3,4,03-11-2019,2.67,5,270
3,9473906,Lovely Studio in Palermo,25602761,Mariano,Recoleta,-34.59395,-58.41423,Entire home/apt,3,43,21-11-2019,0.89,1,142
4,34155238,Cozy and comfortable apartment in Belgrano.,128766227,Maite,Colegiales,-34.56911,-58.45162,Entire home/apt,2,13,10-11-2019,2.12,1,241


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6864 entries, 0 to 6863
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              6864 non-null   int64  
 1   name                            6862 non-null   object 
 2   host_id                         6864 non-null   int64  
 3   host_name                       6854 non-null   object 
 4   neighbourhood                   6864 non-null   object 
 5   latitude                        6864 non-null   float64
 6   longitude                       6864 non-null   float64
 7   room_type                       6864 non-null   object 
 8   minimum_nights                  6864 non-null   int64  
 9   number_of_reviews               6864 non-null   int64  
 10  last_review                     4969 non-null   object 
 11  reviews_per_month               4969 non-null   float64
 12  calculated_host_listings_count  68

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,15843708,Monoambiente en Barrio Norte,19787638,Pablo,Recoleta,-34.59053,-58.40898,Entire home/apt,2,26,03-11-2019,0.90,1,74
1,9735218,Busco Roomate :),38507726,Angela,Palermo,-34.58633,-58.41312,Private room,1,0,NaN,NaN,1,0
2,35682605,Betty´s home,100972248,Cecilia,Balvanera,-34.59979,-58.39340,Entire home/apt,3,4,03-11-2019,2.67,5,270
3,9473906,Lovely Studio in Palermo,25602761,Mariano,Recoleta,-34.59395,-58.41423,Entire home/apt,3,43,21-11-2019,0.89,1,142
4,34155238,Cozy and comfortable apartment in Belgrano.,128766227,Maite,Colegiales,-34.56911,-58.45162,Entire home/apt,2,13,10-11-2019,2.12,1,241


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6864 entries, 0 to 6863
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              6864 non-null   int64  
 1   name                            6862 non-null   object 
 2   host_id                         6864 non-null   int64  
 3   host_name                       6854 non-null   object 
 4   neighbourhood                   6864 non-null   object 
 5   latitude                        6864 non-null   float64
 6   longitude                       6864 non-null   float64
 7   room_type                       6864 non-null   object 
 8   minimum_nights                  6864 non-null   int64  
 9   number_of_reviews               6864 non-null   int64  
 10  last_review                     4969 non-null   object 
 11  reviews_per_month               4969 non-null   float64
 12  calculated_host_listings_count  68

## Preprocesamiento

In [18]:
df_preprocessed_train, train_price_map = preprocess_data(df_raw_train)

df_pre_IQR = IQR_outlier_removal(df_preprocessed_train.copy(), 
                                 [ "price_log", "minimum_nights_log", "number_of_reviews_log" ,"reviews_per_month", "availability_365_log", "calculated_host_listings_count_log" , "days_since_last_review"])

df_pre_win = winsorize_outlier_removal(df_preprocessed_train.copy(), 
                                       [ "price_log", "minimum_nights_log", "number_of_reviews_log" ,"reviews_per_month", "availability_365_log", "calculated_host_listings_count_log" , "days_since_last_review"])

### Gráfica de los Outliers

In [19]:
tbl_paths = os.path.join(tex_folder_path, "tables");
percent_lst = [0, 0.15, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99] 

generar_tabla_descriptiva_latex(
    df=df_preprocessed_train,
    cols=[ "price_log", "minimum_nights_log", "number_of_reviews_log", "reviews_per_month", "availability_365_log", "calculated_host_listings_count_log" , "days_since_last_review"],
    percentiles=percent_lst,
    output_path= tbl_paths,
    filename="df_prepocessed_train_desc.tex",
    caption=r"Datos estadísticos pre-procesados del conjunto de entrenamiento.",
    label=r"tab:df-preprocessed-train-desc"
)
 
generar_tabla_descriptiva_latex(
    df=df_pre_IQR,
    cols=[ "price_log", "minimum_nights_log", "number_of_reviews_log", "reviews_per_month", "availability_365_log", "calculated_host_listings_count_log" , "days_since_last_review"],
    percentiles=percent_lst,
    output_path= tbl_paths,
    filename="df_iqr_desc.tex",
    caption=r"Estadísticos descriptivos de las variables numéricas tras el tratamiento de outliers con IQR (además de aplicar \lstinline[style=python]{np.expm1} a \texttt{price}).",
    label=r"tab:df-iqr-desc-log"
)

generar_tabla_descriptiva_latex(
    df=df_raw_train,
    cols=["price","minimum_nights","number_of_reviews","reviews_per_month","calculated_host_listings_count","availability_365","days_since_last_review"],
    percentiles=percent_lst,
    output_path= tbl_paths,
    filename="df_raw_train_desc.tex",
    caption=r"Datos estadísticos originales del conjunto de entrenamiento.",
    label=r"tab:df-raw-train-desc"
)

Archivo LaTeX generado: Informe/tex/tables/df_prepocessed_train_desc.tex
Archivo LaTeX generado: Informe/tex/tables/df_iqr_desc.tex
Archivo LaTeX generado: Informe/tex/tables/df_raw_train_desc.tex


In [20]:
generar_histogramas(df_raw_train, "raw_train", img_folder_path)
generar_histogramas(df_preprocessed_train, "preprocessed_train", img_folder_path)
generar_histogramas(df_pre_IQR, "pre_IQR", img_folder_path)

→ Guardado: Informe/img/histogramas/id_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/host_id_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/host_id_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/latitude_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/latitude_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/longitude_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/longitude_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/price_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/price_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/minimum_nights_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/minimum_nights_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/number_of_reviews_raw_train_histograma_bins50.png
→ Guardado: Informe/img/histogramas/number_of_reviews_raw_train_his

In [21]:
dfs = [df_preprocessed_train, df_pre_win, df_pre_IQR]
labels = ["Original (sin tratamiento de outliers)", "Winsorize (1–99%)", "IQR"]

plot_percentiles_all_methods([ "price_log", "minimum_nights_log", "number_of_reviews_log", "reviews_per_month", "availability_365_log", "calculated_host_listings_count_log" , "days_since_last_review"], dfs, labels, percentiles= [0,10, 25, 50, 75, 90, 95, 100])
plot_percentiles_all_methods([ "price_log", "minimum_nights_log", "number_of_reviews_log", "reviews_per_month", "availability_365_log", "calculated_host_listings_count_log" , "days_since_last_review"], dfs, labels, percentiles=[95, 96, 97, 98, 99, 99.5, 100])
plot_percentiles_all_methods([ "price_log", "minimum_nights_log", "number_of_reviews_log", "reviews_per_month", "availability_365_log", "calculated_host_listings_count_log" , "days_since_last_review"], dfs, labels, percentiles=[0, 0.5 , 1, 2, 3, 4, 5, 10 ,12])


Guardado: Informe/img/percentiles/price_log_percentiles_max=100_min=0.png
Guardado: Informe/img/percentiles/minimum_nights_log_percentiles_max=100_min=0.png
Guardado: Informe/img/percentiles/minimum_nights_log_percentiles_max=100_min=0.png
Guardado: Informe/img/percentiles/number_of_reviews_log_percentiles_max=100_min=0.png
Guardado: Informe/img/percentiles/number_of_reviews_log_percentiles_max=100_min=0.png
Guardado: Informe/img/percentiles/reviews_per_month_percentiles_max=100_min=0.png
Guardado: Informe/img/percentiles/reviews_per_month_percentiles_max=100_min=0.png
Guardado: Informe/img/percentiles/availability_365_log_percentiles_max=100_min=0.png
Guardado: Informe/img/percentiles/availability_365_log_percentiles_max=100_min=0.png
Guardado: Informe/img/percentiles/calculated_host_listings_count_log_percentiles_max=100_min=0.png
Guardado: Informe/img/percentiles/calculated_host_listings_count_log_percentiles_max=100_min=0.png
Guardado: Informe/img/percentiles/days_since_last_review

## Split the data

Split the DataFrame into training and validation sets (70% for training, 30% for validation).


In [22]:
X = df_pre_IQR.drop("price_log", axis=1)
y = df_pre_IQR["price_log"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
print("Tamaños:", X_train.shape, X_val.shape)

Tamaños: (11209, 17) (4804, 17)


# Define models

In [23]:
# Identificamos variables numéricas y categóricas
num_features = X.select_dtypes(include=["int64", "float64"]).columns
cat_features = X.select_dtypes(include=["object"]).columns

# Definimos transformaciones
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ]
)

In [24]:
models = {
    "Dummy": DummyRegressor(strategy="mean"),
    "LinearRegression": LinearRegression(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "NeuralNetwork": MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
}

# Train the models


In [25]:
# ---- Ejecutar evaluación ----
results_df = evaluar_modelos(models, preprocessor, X_train, y_train, X_val, y_val)
display(results_df)

# ---- Exportar a LaTeX ----
export_df_to_latex(
    df=results_df,
    filename="model_results_final.tex",
    caption="Resultados del primer barrido de modelos sobre la partición de validación (70/30).",
    label="tab:model-results-first-barrido",
    base_path="Informe/tex",
    folder_name="tables"
)

Dummy                RMSE=  0.732 | MSE=  0.535 | MAE=  0.572 | R2= -0.000
LinearRegression     RMSE=  0.586 | MSE=  0.344 | MAE=  0.445 | R2=  0.358
LinearRegression     RMSE=  0.586 | MSE=  0.344 | MAE=  0.445 | R2=  0.358
DecisionTree         RMSE=  0.801 | MSE=  0.641 | MAE=  0.609 | R2= -0.198
DecisionTree         RMSE=  0.801 | MSE=  0.641 | MAE=  0.609 | R2= -0.198
RandomForest         RMSE=  0.553 | MSE=  0.306 | MAE=  0.420 | R2=  0.428
RandomForest         RMSE=  0.553 | MSE=  0.306 | MAE=  0.420 | R2=  0.428
GradientBoosting     RMSE=  0.565 | MSE=  0.319 | MAE=  0.431 | R2=  0.404
GradientBoosting     RMSE=  0.565 | MSE=  0.319 | MAE=  0.431 | R2=  0.404
NeuralNetwork        RMSE=  0.601 | MSE=  0.361 | MAE=  0.456 | R2=  0.326
NeuralNetwork        RMSE=  0.601 | MSE=  0.361 | MAE=  0.456 | R2=  0.326


,Modelo,RMSE,MSE,MAE,R^{2}
0,Dummy,0.731699,0.535383,0.572285,-0.000105
1,LinearRegression,0.586430,0.343900,0.445244,0.357589
2,DecisionTree,0.800934,0.641495,0.608925,-0.198324
3,RandomForest,0.553297,0.306138,0.419915,0.428129
4,GradientBoosting,0.564862,0.319069,0.431338,0.403974
5,NeuralNetwork,0.600613,0.360737,0.455906,0.326138


→ LaTeX generado: Informe/tex/tables/model_results_final.tex


## Evaluate the models

Evaluate the performance of models on the validation data using appropriate metrics.


In [26]:
optimized_models = {}

In [27]:
lasso_result = tune_and_report(
    name="Lasso",
    model=Lasso(max_iter=10000, random_state=42),
    param_grid={"model__alpha": [0.001, 0.01, 0.1, 1, 10, 100]},
    preprocessor=preprocessor,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    export_latex=True,
    latex_filename="Lasso_gridsearch_results.tex",
    latex_caption="Resultados del GridSearchCV para la regresión Lasso.",
    latex_label="tab:lasso-gridsearch"
)

# Best
optimized_models["Lasso Optimizado"] = lasso_result["best_estimator"]


🔍 Optimizando Lasso...

👉 Mejor params: {'model__alpha': 0.001}
👉 Mejor RMSE (CV): 0.589769904806074
📌 Lasso (validación) -> RMSE=0.584, MSE=0.342, MAE=0.445, R2=0.362
→ LaTeX generado: Informe/tex/tables/Lasso_gridsearch_results.tex
👉 Mejor params: {'model__alpha': 0.001}
👉 Mejor RMSE (CV): 0.589769904806074
📌 Lasso (validación) -> RMSE=0.584, MSE=0.342, MAE=0.445, R2=0.362
→ LaTeX generado: Informe/tex/tables/Lasso_gridsearch_results.tex


In [28]:
ridge_result = tune_and_report(
    name="Ridge",
    model=Ridge(max_iter=10000, random_state=42),
    param_grid={"model__alpha": [0.001, 0.01, 0.1, 1, 10, 100]},
    preprocessor=preprocessor,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    export_latex=True,
    latex_filename="Ridge_gridsearch_results.tex",
    latex_caption="Resultados del GridSearchCV para regresión Ridge.",
    latex_label="tab:ridge-gridsearch"
)

# Best
optimized_models["Ridge Optimizado"] = ridge_result["best_estimator"]


🔍 Optimizando Ridge...

👉 Mejor params: {'model__alpha': 0.1}
👉 Mejor RMSE (CV): 0.5883850160956631
📌 Ridge (validación) -> RMSE=0.586, MSE=0.344, MAE=0.445, R2=0.358
→ LaTeX generado: Informe/tex/tables/Ridge_gridsearch_results.tex
👉 Mejor params: {'model__alpha': 0.1}
👉 Mejor RMSE (CV): 0.5883850160956631
📌 Ridge (validación) -> RMSE=0.586, MSE=0.344, MAE=0.445, R2=0.358
→ LaTeX generado: Informe/tex/tables/Ridge_gridsearch_results.tex


In [41]:
gb_result = tune_and_report(
    name="GradientBoosting",
    model=GradientBoostingRegressor(random_state=42),
    param_grid={
        "model__n_estimators": [50, 100, 200],
        "model__learning_rate": [0.01, 0.05, 0.1],
        "model__max_depth": [3, 5, 7],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2]
    },
    preprocessor=preprocessor,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    export_latex=True,
    latex_filename="GradientBoosting_gridsearch_results.tex",
    latex_caption="Resultados del GridSearchCV para Gradient Boosting Regressor.",
    latex_label="tab:gradient-boosting-gridsearch",
    bigTable=True
)

# Best
optimized_models["GradientBoosting Optimizado"] = gb_result["best_estimator"]


🔍 Optimizando GradientBoosting...

👉 Mejor params: {'model__learning_rate': 0.05, 'model__max_depth': 7, 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 200}
👉 Mejor RMSE (CV): 0.5665823848753672
📌 GradientBoosting (validación) -> RMSE=0.556, MSE=0.309, MAE=0.421, R2=0.423
→ LaTeX generado: Informe/tex/tables/GradientBoosting_gridsearch_results.tex
👉 Mejor params: {'model__learning_rate': 0.05, 'model__max_depth': 7, 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 200}
👉 Mejor RMSE (CV): 0.5665823848753672
📌 GradientBoosting (validación) -> RMSE=0.556, MSE=0.309, MAE=0.421, R2=0.423
→ LaTeX generado: Informe/tex/tables/GradientBoosting_gridsearch_results.tex


In [29]:
rf_result = tune_and_report(
    name="RandomForest",
    model=RandomForestRegressor(random_state=42),
    param_grid={
        "model__n_estimators": [1, 50, 100, 200],
        "model__max_depth": [1, 3, 5, 10, 20, None],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2],
    },
    preprocessor=preprocessor,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    export_latex=True,
    latex_filename="RandomForest_gridsearch_results.tex",
    latex_caption="GridSearch para Random Forest.",
    latex_label="tab:random-forest",
    bigTable=True
)

# Best
optimized_models["RandomForest Optimizado"] = rf_result["best_estimator"]



🔍 Optimizando RandomForest...

👉 Mejor params: {'model__max_depth': 20, 'model__min_samples_leaf': 2, 'model__min_samples_split': 5, 'model__n_estimators': 200}
👉 Mejor RMSE (CV): 0.5702863545964417
📌 RandomForest (validación) -> RMSE=0.552, MSE=0.305, MAE=0.419, R2=0.431
→ LaTeX generado: Informe/tex/tables/RandomForest_gridsearch_results.tex
👉 Mejor params: {'model__max_depth': 20, 'model__min_samples_leaf': 2, 'model__min_samples_split': 5, 'model__n_estimators': 200}
👉 Mejor RMSE (CV): 0.5702863545964417
📌 RandomForest (validación) -> RMSE=0.552, MSE=0.305, MAE=0.419, R2=0.431
→ LaTeX generado: Informe/tex/tables/RandomForest_gridsearch_results.tex


In [30]:
dt_result = tune_and_report(
    name="DecisionTreeRegressor",
    model=DecisionTreeRegressor(random_state=42),
    param_grid={
        "model__max_depth": [1, 2, 3, 5, 10, 20, None],
        "model__min_samples_split": [2, 5, 10],
        "model__min_samples_leaf": [1, 2, 4],
    },
    preprocessor=preprocessor,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    export_latex=True,
    latex_filename="DecisionTreeRegressor_gridsearch_results.tex",
    latex_caption="GridSearch para Decision Tree.",
    latex_label="tab:decision-tree",
    bigTable=True
)

# Best
optimized_models["DecisionTreeRegressor Optimizado"] = dt_result["best_estimator"] 


🔍 Optimizando DecisionTreeRegressor...

👉 Mejor params: {'model__max_depth': 5, 'model__min_samples_leaf': 4, 'model__min_samples_split': 2}
👉 Mejor RMSE (CV): 0.6078622182068706
📌 DecisionTreeRegressor (validación) -> RMSE=0.600, MSE=0.360, MAE=0.458, R2=0.327
→ LaTeX generado: Informe/tex/tables/DecisionTreeRegressor_gridsearch_results.tex
👉 Mejor params: {'model__max_depth': 5, 'model__min_samples_leaf': 4, 'model__min_samples_split': 2}
👉 Mejor RMSE (CV): 0.6078622182068706
📌 DecisionTreeRegressor (validación) -> RMSE=0.600, MSE=0.360, MAE=0.458, R2=0.327
→ LaTeX generado: Informe/tex/tables/DecisionTreeRegressor_gridsearch_results.tex


In [31]:
nn_result = tune_and_report(
    name="NeuralNetwork",
    model=MLPRegressor(
        max_iter=500,
        random_state=42,
        early_stopping=True
    ),
    param_grid={
        "model__hidden_layer_sizes": [(64,), (128, 64), (64, 32)],
        "model__alpha": [0.0001, 0.001, 0.01],
        "model__learning_rate_init": [0.001, 0.01],
    },
    preprocessor=preprocessor,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,

    # export LaTeX
    export_latex=True,
    latex_filename="MLP_gridsearch_results.tex",
    latex_caption="Resultados del GridSearchCV para el modelo MLPRegressor (Neural Network).",
    latex_label="tab:mlpregressor-gridsearch",
    bigTable=True  # porque las NN suelen tener muchas combinaciones
)

# Guardar el mejor modelo optimizado
optimized_models["NeuralNetwork Optimizado"] = nn_result["best_estimator"]



🔍 Optimizando NeuralNetwork...

👉 Mejor params: {'model__alpha': 0.0001, 'model__hidden_layer_sizes': (128, 64), 'model__learning_rate_init': 0.001}
👉 Mejor RMSE (CV): 0.5947026743635637
📌 NeuralNetwork (validación) -> RMSE=0.581, MSE=0.337, MAE=0.445, R2=0.370
→ LaTeX generado: Informe/tex/tables/MLP_gridsearch_results.tex
👉 Mejor params: {'model__alpha': 0.0001, 'model__hidden_layer_sizes': (128, 64), 'model__learning_rate_init': 0.001}
👉 Mejor RMSE (CV): 0.5947026743635637
📌 NeuralNetwork (validación) -> RMSE=0.581, MSE=0.337, MAE=0.445, R2=0.370
→ LaTeX generado: Informe/tex/tables/MLP_gridsearch_results.tex


## Corss Validation

In [42]:
all_models = {
    **models,
    **optimized_models
}

In [43]:
cv_results_df = cross_validate_models(
    models_dict=all_models,
    X=X,
    y=y,
    preprocessor=preprocessor,
    cv_folds=5,
    export_latex=True,
    latex_filename="cv_results_all_models.tex",
    latex_caption="Validación cruzada (5-Fold) para todos los modelos optimizados.",
    latex_label="tab:cv-all-models"
)

→ Ejecutando CV para: Dummy
→ Ejecutando CV para: LinearRegression
→ Ejecutando CV para: LinearRegression
→ Ejecutando CV para: DecisionTree
→ Ejecutando CV para: DecisionTree
→ Ejecutando CV para: RandomForest
→ Ejecutando CV para: RandomForest
→ Ejecutando CV para: GradientBoosting
→ Ejecutando CV para: GradientBoosting
→ Ejecutando CV para: NeuralNetwork
→ Ejecutando CV para: NeuralNetwork
→ Ejecutando CV para: Lasso Optimizado
→ Ejecutando CV para: Lasso Optimizado
→ Ejecutando CV para: Ridge Optimizado
→ Ejecutando CV para: Ridge Optimizado
→ Ejecutando CV para: RandomForest Optimizado
→ Ejecutando CV para: RandomForest Optimizado
→ Ejecutando CV para: DecisionTreeRegressor Optimizado
→ Ejecutando CV para: DecisionTreeRegressor Optimizado
→ Ejecutando CV para: NeuralNetwork Optimizado
→ Ejecutando CV para: NeuralNetwork Optimizado
→ Ejecutando CV para: GradientBoosting Optimizado
→ Ejecutando CV para: GradientBoosting Optimizado


,Modelo,rmse_mean,rmse_std,MSE_mean,MAE_mean,R2_mean
11,GradientBoosting Optimizado,0.559039,0.008636,0.312599,0.422906,0.419446
8,RandomForest Optimizado,0.560440,0.009807,0.314189,0.423249,0.416497
3,RandomForest,0.562411,0.009469,0.316396,0.424902,0.412375
4,GradientBoosting,0.571912,0.007864,0.327145,0.436553,0.392437
10,NeuralNetwork Optimizado,0.584877,0.007949,0.342144,0.446510,0.364467
7,Ridge Optimizado,0.587532,0.006681,0.345238,0.447932,0.358746
1,LinearRegression,0.587540,0.006684,0.345248,0.447910,0.358727
6,Lasso Optimizado,0.588357,0.006571,0.346208,0.449443,0.356948
9,DecisionTreeRegressor Optimizado,0.602546,0.005816,0.363096,0.459290,0.325499
5,NeuralNetwork,0.617816,0.009945,0.381796,0.472082,0.290335


→ LaTeX generado: Informe/tex/tables/cv_results_all_models.tex


# Load the test data and save results

In [44]:
# --- Carga y Preprocesamiento de Test ---
# Pasamos el mapa de precios calculado en train para evitar data leakage y errores
df_pre_test, _ = preprocess_data(df_raw_test, price_map=train_price_map)

# Aplicamos el mismo tratamiento de outliers (clipping) usando las columnas definidas
# Nota: Idealmente deberíamos usar los límites de train, pero IQR_outlier_removal usa los del propio DF.
# Para consistencia con el código previo, lo mantenemos, pero aseguramos pasar las columnas.
df_pre_test_IQR = IQR_outlier_removal(df_pre_test.copy(), [ "price_log", "minimum_nights_log", "number_of_reviews_log" ,"reviews_per_month", "availability_365_log", "calculated_host_listings_count_log" , "days_since_last_review"])

# --- Generación de Predicciones ---

if is_Drive:
    pred_folder = os.path.join(base_path, "pred")
else:
    pred_folder = "pred"
os.makedirs(pred_folder, exist_ok=True)

# Definir X e y completos para re-entrenamiento final
# Usamos df_pre_IQR (train) para entrenar
target_col = "price_log"
if target_col not in df_pre_IQR.columns:
    raise ValueError(f"La columna objetivo '{target_col}' no está en el DataFrame de entrenamiento.")

X_full = df_pre_IQR.drop(columns=[target_col], errors='ignore')
y_full = df_pre_IQR[target_col]

# Alinear columnas de test con train (asegurar mismo orden y columnas)
# Rellenar columnas faltantes con 0 si las hubiera (ej. one-hot encoding diferencias)
for col in X_full.columns:
    if col not in df_pre_test_IQR.columns:
        df_pre_test_IQR[col] = 0
        
test_df_final = df_pre_test_IQR[X_full.columns]

print(f"Entrenando modelos finales con {X_full.shape[0]} muestras y generando predicciones...")

# Guardamos los IDs para el archivo de submission
test_ids = df_raw_test["id"]

for name, model in all_models.items():
    print(f"→ Procesando {name}...")
    
    # Crear pipeline si no lo es
    if isinstance(model, Pipeline):
        pipe = model
    else:
        pipe = Pipeline([("preprocessor", preprocessor), ("model", model)])
    
    # Re-entrenar con TODO el dataset (Train + Val)
    pipe.fit(X_full, y_full)
    
    # Predecir
    y_pred_log = pipe.predict(test_df_final)
    y_pred = np.expm1(y_pred_log) # Revertir log
    
    # Guardar
    sub = pd.DataFrame({"id": test_ids, "price": y_pred})
    fname = f"pred_{name}.csv"
    sub.to_csv(os.path.join(pred_folder, fname), index=False)
    print(f"   Guardado: {fname}")

print("\n¡Todas las predicciones han sido generadas exitosamente!")

Entrenando modelos finales con 16013 muestras y generando predicciones...
→ Procesando Dummy...
   Guardado: pred_Dummy.csv
→ Procesando LinearRegression...
   Guardado: pred_LinearRegression.csv
→ Procesando DecisionTree...
   Guardado: pred_LinearRegression.csv
→ Procesando DecisionTree...
   Guardado: pred_DecisionTree.csv
→ Procesando RandomForest...
   Guardado: pred_DecisionTree.csv
→ Procesando RandomForest...
   Guardado: pred_RandomForest.csv
→ Procesando GradientBoosting...
   Guardado: pred_RandomForest.csv
→ Procesando GradientBoosting...
   Guardado: pred_GradientBoosting.csv
→ Procesando NeuralNetwork...
   Guardado: pred_GradientBoosting.csv
→ Procesando NeuralNetwork...
   Guardado: pred_NeuralNetwork.csv
→ Procesando Lasso Optimizado...
   Guardado: pred_NeuralNetwork.csv
→ Procesando Lasso Optimizado...
   Guardado: pred_Lasso Optimizado.csv
→ Procesando Ridge Optimizado...
   Guardado: pred_Lasso Optimizado.csv
→ Procesando Ridge Optimizado...
   Guardado: pred_Ridge

# Entorno Utilizado:

In [45]:
import sys
import platform
import psutil
import importlib.metadata as metadata
import GPUtil
import os

def obtener_info_python():
    info = {
        "Intérprete": sys.executable,
        "Versión de Python": sys.version,
        "Versión de pip": metadata.version("pip")
    }
    return info

def obtener_librerias_instaladas():
    return {pkg.metadata['Name']: pkg.version for pkg in metadata.distributions()}

def obtener_info_sistema():
    info = {
        "Sistema Operativo": platform.system(),
        "Versión OS": platform.version(),
        "Nombre del SO": platform.platform(),
        "Arquitectura": platform.architecture(),
        "CPU": platform.processor(),
        "Núcleos (Físicos)": psutil.cpu_count(logical=False),
        "Núcleos (Lógicos)": psutil.cpu_count(logical=True),
        "RAM Total": round(psutil.virtual_memory().total / (1024 ** 3), 2),  # En GB
    }
    return info

def obtener_info_gpu():
    try:
        gpus = GPUtil.getGPUs()
        if gpus:
            return [{"Nombre": gpu.name, "Memoria": gpu.memoryTotal, "Uso": gpu.memoryUsed} for gpu in gpus]
        else:
            return "No se encontró GPU"
    except Exception:
        return "Error al obtener info de GPU"

def generar_reporte():
    lines = []
    lines.append("=== Información de Python ===")
    for clave, valor in obtener_info_python().items():
        lines.append(f"{clave}: {valor}")
        
    lines.append("\n=== Librerías instaladas ===")
    # Ordenamos alfabéticamente para mayor claridad
    libs = obtener_librerias_instaladas()
    for nombre in sorted(libs.keys()):
        lines.append(f"{nombre}: {libs[nombre]}")
        
    lines.append("\n=== Información del sistema ===")
    for clave, valor in obtener_info_sistema().items():
        lines.append(f"{clave}: {valor}")
        
    lines.append("\n=== Información de GPU ===")
    lines.append(str(obtener_info_gpu()))
    
    return "\n".join(lines)

def guardar_info_txt(filename="system_info.txt"):
    reporte = generar_reporte()
    
    # Imprimir en consola
    print(reporte)
    
    # Guardar en archivo
    with open(filename, "w", encoding="utf-8") as f:
        f.write(reporte)
    
    print(f"\n[INFO] Información exportada exitosamente a: {os.path.abspath(filename)}")

if __name__ == "__main__":
    guardar_info_txt("Informe/code/system_info.txt")

=== Información de Python ===
Intérprete: /mnt/d/git/OBL MACHINE/.venv/bin/python
Versión de Python: 3.12.3 (main, Nov  6 2025, 13:44:16) [GCC 13.3.0]
Versión de pip: 25.3

=== Librerías instaladas ===
GPUtil: 1.4.0
Jinja2: 3.1.6
MarkupSafe: 3.0.3
PyYAML: 6.0.3
Pygments: 2.19.2
Unidecode: 1.4.0
asttokens: 3.0.1
autocommand: 2.2.2
backports.tarfile: 1.2.0
certifi: 2025.11.12
charset-normalizer: 3.4.4
comm: 0.2.3
contourpy: 1.3.3
cycler: 0.12.1
debugpy: 1.8.17
decorator: 5.2.1
executing: 2.2.1
filelock: 3.20.0
fonttools: 4.61.0
fsspec: 2025.12.0
hf-xet: 1.2.0
huggingface-hub: 0.36.0
idna: 3.11
importlib_metadata: 8.0.0
inflect: 7.3.1
ipykernel: 7.1.0
ipython: 9.8.0
ipython_pygments_lexers: 1.1.1
ipywidgets: 8.1.8
jaraco.collections: 5.1.0
jaraco.context: 5.3.0
jaraco.functools: 4.0.1
jaraco.text: 3.12.1
jedi: 0.19.2
joblib: 1.5.2
jupyter_client: 8.6.3
jupyter_core: 5.9.1
jupyterlab_widgets: 3.0.16
kiwisolver: 1.4.9
matplotlib: 3.10.7
matplotlib-inline: 0.2.1
more-itertools: 10.3.0
mpmath